## Thu thập dữ liệu bằng parse HTML - Playlists_data
- Họ tên: Nguyễn Minh Phúc
- MSSV: 19120119

## Môi trường code
Dùng phiên bản các package như trong file "min_ds-env.yml"


***
## Import

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import pandas as pd

Công việc trong phần này là viết hàm `collect_playlist` ở bên dưới. Hàm này dùng để thu thập dữ liệu của playlist trên Soundcloud. Hàm này dùng các input như sau:
 - playlist_urls_file: là chuỗi, cho biết tên của file txt chứa các đường link đến tất cả các playlist, mỗi đường link nằm trên một dòng.
 - playlists_file: là chuỗi , chúng ta sẽ viết danh sách các playlist vào file này. 
 
Chúng ta sẽ sử dụng các biến như sau :
 - urls_file: là danh sách link trong file playlist_urls_file.txt
 - num_of_records : số lượng records
 - url : link dẫn tới playlist
 - driver: khởi tạo driver , chúng ta sẽ truyền vào đường dẫn file `chromedriver.exe` . Sẽ cần dowload bản [chromedriver.exe](https://chromedriver.chromium.org/downloads?fbclid=IwAR2J5mdJzt-p7RA28YebsAVB2-hrxhdY2PXfU8kYBKwoBCK-HtH-XDI8T_s) phù hợp với bản chrome mình đang sử dụng.
 - name: tên hiện thị của playlist
 - num_tracks: số lượng bài hát có trong mỗi playlist
 - duration: Tổng số thời gian phát của cả platlist
 - owner: người phát hành playlist
 - likes: Số lượt like của playlist
 - num_reposts: Số lượng bản đăng lại
 - tag: nhãn của playlist (#...)
 - posted_on: ngày đăng playlist

In [2]:
def collect_playlist(playlist_urls_file, playlists_file):
    # YOUR CODE HERE
    urls_file = open(playlist_urls_file, "r")    
    fout = open(playlists_file, "w", encoding="utf-8")
    fout.write("url\tname\tnum_tracks\tduration\towner\tlikes\tnum_reposts\ttag\tposted_on\n")
            
    options = Options()
    options.headless = True
    driver = webdriver.Chrome('chromedriver.exe', options=options)
    
    num_of_records = 0
    
    for url in urls_file:
        driver.get(url)        
        driver.implicitly_wait(0.5)
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # url
        fout.write(url.strip() + '\t')
        # name
        name = soup.find('h1', class_='soundTitle__title')
        if name is None:
            fout.write('None\t')
        else: fout.write(name.text.strip() + '\t')
            
        # num_tracks
        num_tracks = soup.find('div', class_='genericTrackCount__title')
        if num_tracks is None:
            fout.write('0\t')
        else: fout.write(num_tracks.text + '\t')
            
        # duration
        duration = soup.find('div', class_='genericTrackCount__duration')
        if duration is None:
            fout.write('0\t')
        else: fout.write(duration.text + '\t')
            
        # owner
        owner = soup.find('h2', class_='soundTitle__username')
        if owner is None:
            fout.write('None\t')
        else: fout.write(owner.text.strip() + '\t')
        
        # likes
        likes = soup.select('a.sc-ministats-likes span[aria-hidden="true"]')
        if len(likes) == 0:
            fout.write('0\t')
        else: fout.write(likes[0].text + '\t')
        
        # num_reposts
        num_reposts = soup.select('a.sc-ministats-reposts span[aria-hidden="true"]')
        if len(num_reposts) == 0:
            fout.write('0\t')
        else: fout.write(num_reposts[0].text + '\t')
        
        # tag
        tag = soup.find('span', class_='sc-truncate sc-tagContent')
        if tag is None:
            fout.write('None\t')
        else:
            fout.write(tag.text + '\t')
            
        # posted_on
        time = soup.find('time', class_='relativeTime')
        if time is None:
            fout.write('None\n')
        else: fout.write(time['title'] + '\n')
        
        num_of_records += 1
        if num_of_records == 1010:
            break
        ###
                
    urls_file.close()
    fout.close()
    driver.quit()    

## Tiến hành crawl dữ liệu
Vì thời gian crawl dữ liệu khá lâu (khoảng tầm 30 phút) nên nếu muốn thực thi hàm trên, vui lòng uncomment cell bên dưới để chạy và nên backup file dữ liệu đã crawl

In [3]:
#collect_playlist('processed-playlists-urls.txt', 'playlists.csv')

## Đọc dữ liệu từ file

In [4]:
courses = pd.read_csv('playlists.csv', sep='\t')
courses

,url,name,num_tracks,duration,owner,likes,num_reposts,tag,posted_on
0,https://soundcloud.com/digitalstreams/sets/dee...,Best of Deep House (Vol. 02),39,1:16:39,Westcoast Radio (EDM),181K,"9,823",Dance & EDM,2 July 2017
1,https://soundcloud.com/courtney-mccann/sets/edm,Edm,469,473:40:13,Courtney Mccann,"2,327",224,None,24 June 2016
2,https://soundcloud.com/digitalstreams/sets/ilo...,Best of House Music (Vol. 01),40,3:23:50,Westcoast Radio (EDM),89.6K,"5,225",Dance & EDM,22 February 2015
3,https://soundcloud.com/digitalstreams/sets/chi...,Best Chill Trap,35,2:15:14,Westcoast Radio (EDM),92.6K,"6,211",Dance & EDM,26 August 2017
4,https://soundcloud.com/digitalstreams/sets/sax...,Best of Saxxy House (2021),36,2:46:58,Westcoast Radio (EDM),75.8K,"4,177",chill,26 October 2015
...,...,...,...,...,...,...,...,...,...
1005,https://soundcloud.com/junaidafghani/sets/edm,EDM,220,28:45:53,JunaidAfghani,1,0,Electronic,12 February 2020
1006,https://soundcloud.com/diverse_sounds20/sets/s...,EDM,188,1:45:37,Diverse Sounds 🍉,0,0,EDM,6 May 2017
1007,https://soundcloud.com/peanutman/sets/spinnin,EDM,172,23:26:10,Jefrey,2,1,Electronic,2 December 2014
1008,https://soundcloud.com/kristyvogelsang/sets/de...,edm,158,54:38,kristy vogelsang,2,1,Dance & EDM,7 January 2016
